## This notebook accompanies our submission for SWAT4HCLS 2023 "data-in-use", showcasing federated queries across UniProt, OMA and Bgee for identifying essential proteins in Yeast

Towards reproducing the analysis in the paper:
Zhang, Xue, Wangxin Xiao, and Xihao Hu. "Predicting essential proteins by integrating orthology, gene expressions, and PPI networks." PloS one 13.4 (2018): e0195410.

#### Note: we build the query progressively to illustrate step by step how to achieve the end result. We also highlight lessons learned for each of the steps, where relevant.

In [14]:
# helper functions
import sys
!{sys.executable} -m pip install SPARQLWrapper
from SPARQLWrapper import SPARQLWrapper, JSON
import sys, os, time
import pandas as pd

# always display full column results (don't truncate output)
pd.set_option('display.max_colwidth', -1)

OMA_SPARQL_endpoint = "https://sparql.omabrowser.org/sparql"
Bgee_SPARQL_endpoint = "https://bgee.org/sparql/"
UniProt_SPARQL_endpoint = "https://sparql.uniprot.org/sparql/"

# the endpoints must be defined as wrappers for executing SPARQL queries
sparql_endpoint_OMA = SPARQLWrapper(OMA_SPARQL_endpoint)
sparql_endpoint_UniProt = SPARQLWrapper(UniProt_SPARQL_endpoint)

# function to print in a table results of a SPARQL query
def pretty_print(results):
    
    # how to transform SPARQL results into Pandas dataframes
    
    # get header (column names) from results
    header = results["results"]["bindings"][0].keys()

    # display table of results:
    table = []
    
    # the SPARQL JSON results to the query are available in the "results", "bindings" entry:
    for entry in results["results"]["bindings"]:
        # append entries from the results to a regular Python list of rows, which we can then transform to a Pandas DF
        row = [entry[column]["value"] if entry.get(column, None) != None else None for column in header]
        table.append(row)
    df = pd.DataFrame(table, columns=list(header))
    return df



/var/folders/5d/3j6fyxq90gs2qdxgzpzsl1gh0000gp/T/ipykernel_54782/2522515982.py:9: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


### 1. Identify the taxon ID of Yeast ("S. cerevisiae") in OMA RDF

#### First attempt: identify this directly in OMA, searching by scientific name of Yeast (Saccharomyces Cerevisiae)

In [12]:
query_OMA_s_cerevisiae = """
PREFIX oma: <http://omabrowser.org/ontology/oma#>
PREFIX orth: <http://purl.org/net/orth#>
PREFIX sio: <http://semanticscience.org/resource/>
PREFIX lscr: <http://purl.org/lscr#>
PREFIX up: <http://purl.uniprot.org/core/>
PREFIX obo: <http://purl.obolibrary.org/obo/>

select ?taxon ?name where {
    ?taxon  a up:Taxon.
    ?taxon up:scientificName ?name.
    filter (strstarts (lcase(?name), "saccharomyces cerevisiae" ))
}
"""

In [13]:
# set the query to be executed against the OMA endpoint and set the return format to JSON
sparql_endpoint_OMA.setQuery(query_OMA_s_cerevisiae)
sparql_endpoint_OMA.setReturnFormat(JSON)

results = sparql_endpoint_OMA.query().convert()

pretty_print(results)


,taxon,name
0,http://purl.uniprot.org/taxonomy/559292,Saccharomyces cerevisiae (strain ATCC 204508 / S288c)
1,http://purl.uniprot.org/taxonomy/764097,Saccharomyces cerevisiae (strain AWRI796)
2,http://purl.uniprot.org/taxonomy/764099,Saccharomyces cerevisiae (strain VIN 13)
3,http://purl.uniprot.org/taxonomy/764101,Saccharomyces cerevisiae (strain FostersO)


 Since there are multiple such taxa, we need to decide which one is the reference we should use in going forward. For this, since the OMA RDF does not store GO annotations, we will federate with UniProt to compute the number of annotations per taxon. We will consider the reference one to be the highest annotated one.

### 2. Federate with UniProt to identify OMA Yeast taxon with most GO term annotations (this will be the reference one)
Note: despite how few taxa are transmitted from OMA to UniProt, this query takes very long to execute. This is  because all annotations need to be fetched from uniprot and sent back over before computing the aggregation (computing aggregations on the UniProt side fails - we have a similar example using Bgee below)

In [18]:
%%time
query_OMA_UniProt_s_cerevisiae = """
PREFIX oma: <http://omabrowser.org/ontology/oma#>
PREFIX orth: <http://purl.org/net/orth#>
PREFIX sio: <http://semanticscience.org/resource/>
PREFIX lscr: <http://purl.org/lscr#>
PREFIX up: <http://purl.uniprot.org/core/>
PREFIX obo: <http://purl.obolibrary.org/obo/>

select ?taxon (count (distinct ?goTerm) as ?num_annot) where {
    ?taxon  a up:Taxon.
    ?taxon up:scientificName ?name.
    filter (strstarts (lcase(?name), "saccharomyces cerevisiae" ))
    SERVICE <http://sparql.uniprot.org/sparql> {
        Select ?taxon ?goTerm where {
            ?protein a up:Protein .
            ?protein up:classifiedWith ?goTerm .
            ?protein  up:organism ?taxon .
        } 
    }
} group by ?taxon order by desc(?num_annot) # limit 1 -- if we only want to directly fetch the top one

"""

sparql_endpoint_OMA.setQuery(query_OMA_UniProt_s_cerevisiae)
sparql_endpoint_OMA.setReturnFormat(JSON)

results = sparql_endpoint_OMA.query().convert()

pretty_print(results)

CPU times: user 56.6 ms, sys: 14.9 ms, total: 71.5 ms
Wall time: 5min 37s


,taxon,num_annot
0,http://purl.uniprot.org/taxonomy/559292,6417
1,http://purl.uniprot.org/taxonomy/764097,81
2,http://purl.uniprot.org/taxonomy/764099,70
3,http://purl.uniprot.org/taxonomy/764101,60


### 2b. Execute same query at UniProt site (invoking OMA remotely)

For performance comparison, we also execute the same query on the UniProt side directly, where we can then perform aggregations locally.

In [17]:
%%time
query_UniProt_OMA = """
PREFIX oma: <http://omabrowser.org/ontology/oma#>
PREFIX orth: <http://purl.org/net/orth#>
PREFIX sio: <http://semanticscience.org/resource/>
PREFIX lscr: <http://purl.org/lscr#>
PREFIX up: <http://purl.uniprot.org/core/>
PREFIX obo: <http://purl.obolibrary.org/obo/>

SELECT ?taxon (count (distinct ?goTerm) as ?num_annot) where {
  SERVICE<https://sparql.omabrowser.org/sparql>{
select ?taxon where {
?taxon  a up:Taxon.
?taxon up:scientificName ?name.
filter (strstarts (lcase(?name), "saccharomyces cerevisiae" ))
    } }
?protein a up:Protein .
    	?protein up:classifiedWith ?goTerm .
    	?protein  up:organism ?taxon . 
  
} group by ?taxon order by desc(?num_annot) # limit 1 -- if we only want to directly fetch the top one


"""

sparql_endpoint_UniProt.setQuery(query_UniProt_OMA)
sparql_endpoint_UniProt.setReturnFormat(JSON)

results = sparql_endpoint_UniProt.query().convert()

pretty_print(results)

CPU times: user 17.4 ms, sys: 5.18 ms, total: 22.6 ms
Wall time: 1.28 s


,num_annot,taxon
0,6417,http://purl.uniprot.org/taxonomy/559292
1,81,http://purl.uniprot.org/taxonomy/764097
2,70,http://purl.uniprot.org/taxonomy/764099
3,60,http://purl.uniprot.org/taxonomy/764101


The previous query illustrates that the taxon we are looking for is certainly NCBI ID 559292. We will use this throughout the remainder of the tutorial directly.

The difference in execution time between the first variant (5m37s) and the second one (1.2s) is significant.

###### **LESSON LEARNED no. 1: generating and transfering large intermediate results can make order of magnitude difference in performance. However, it is not always easy to predict which subqueries will lead to this problem.**


### 3. Compute orthologs of S. cerevisiae in reference species. 

Here we just illustrate the OMA query required to compute orthologs of S. cerevisiae:

In [21]:
query_OMA_orthologs = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX dc: <http://purl.org/dc/elements/1.1/>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX obo: <http://purl.obolibrary.org/obo/>
PREFIX ensembl: <http://rdf.ebi.ac.uk/resource/ensembl/>
PREFIX oma: <http://omabrowser.org/ontology/oma#>
PREFIX orth: <http://purl.org/net/orth#>
PREFIX sio: <http://semanticscience.org/resource/>
PREFIX taxon: <http://purl.uniprot.org/taxonomy/>
PREFIX up: <http://purl.uniprot.org/core/>
PREFIX void: <http://rdfs.org/ns/void#>
PREFIX lscr: <http://purl.org/lscr#>
PREFIX genex: <http://purl.org/genex#>

select ?p ?species ?orthologGeneEns ?anat  where {
?p a orth:Protein.
?p orth:organism/obo:RO_0002162 taxon:559292.
?cluster a orth:OrthologsCluster.
?cluster orth:hasHomologousMember ?node1.
?cluster orth:hasHomologousMember ?node2. 
?node2 orth:hasHomologousMember* ?ortholog. 
?node1 orth:hasHomologousMember* ?p.
?ortholog sio:SIO_010079 ?gene . #is encoded by
?gene lscr:xrefEnsemblGene ?orthologGeneEns .
?ortholog orth:organism/obo:RO_0002162/up:scientificName ?species.
  
filter(?node1 != ?node2) 
}

"""

### 4. Federate OMA and Bgee to compute orthologs and retrieve anatomical entities (tissues/organs) where these orthologs are highly expressed

In principle, the following query should be used. However, this will likely fail because the result size is too big.

In [106]:
%%time
query_Bgee_OMA = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX dc: <http://purl.org/dc/elements/1.1/>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX obo: <http://purl.obolibrary.org/obo/>
PREFIX ensembl: <http://rdf.ebi.ac.uk/resource/ensembl/>
PREFIX oma: <http://omabrowser.org/ontology/oma#>
PREFIX orth: <http://purl.org/net/orth#>
PREFIX sio: <http://semanticscience.org/resource/>
PREFIX taxon: <http://purl.uniprot.org/taxonomy/>
PREFIX up: <http://purl.uniprot.org/core/>
PREFIX void: <http://rdfs.org/ns/void#>
PREFIX lscr: <http://purl.org/lscr#>
PREFIX genex: <http://purl.org/genex#>

select ?p ?species ?orthologGeneEns ?anat  where {
?p a orth:Protein.
?p orth:organism/obo:RO_0002162 taxon:559292.
?cluster a orth:OrthologsCluster.
?cluster orth:hasHomologousMember ?node1.
?cluster orth:hasHomologousMember ?node2. 
?node2 orth:hasHomologousMember* ?ortholog. 
?node1 orth:hasHomologousMember* ?p.
?ortholog sio:SIO_010079 ?gene . #is encoded by
?gene lscr:xrefEnsemblGene ?orthologGeneEns .
?ortholog orth:organism/obo:RO_0002162/up:scientificName ?species.
  
filter(?node1 != ?node2) 
  
  SERVICE <https://bgee.org/sparql/> {
			SELECT DISTINCT ?orthologGeneEns ?anat {
				?geneB a orth:Gene .
				?geneB lscr:xrefEnsemblGene ?orthologGeneEns .
                ?expr <http://purl.org/genex#hasSequenceUnit> ?geneB.
                ?expr a <http://purl.org/genex#Expression> .
                ?expr genex:hasConfidenceLevel obo:CIO_0000029 . # high confidence level
                ?expr genex:hasExpressionLevel ?exprLevel .
                FILTER (?exprLevel > 99) # highly expressed      
                ?expr genex:hasExpressionCondition ?cond .
                ?cond genex:hasAnatomicalEntity ?anat .
			}
  }
 
}

"""

sparql_endpoint_OMA.setQuery(query_Bgee_OMA)
sparql_endpoint_OMA.setReturnFormat(JSON)

results = sparql_endpoint_OMA.query().convert()

pretty_print(results)

KeyboardInterrupt: 

### As an optimisation, we want to compute *expression breadth* per ortholog by federating with Bgee and performing the aggregation (counting anatomical entities) on the Bgee side. This will significantly reduce the size of results being sent back over (to a single count per orthologous gene).

First (obvious) attempt is to join on the gene ID (here, the Ensembl gene, stored in *?orthologousGeneEns*), which would also be used to group by and count tissues in the Bgee side:

In [205]:
%%time
query_limits = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX dc: <http://purl.org/dc/elements/1.1/>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX obo: <http://purl.obolibrary.org/obo/>
PREFIX ensembl: <http://rdf.ebi.ac.uk/resource/ensembl/>
PREFIX oma: <http://omabrowser.org/ontology/oma#>
PREFIX orth: <http://purl.org/net/orth#>
PREFIX sio: <http://semanticscience.org/resource/>
PREFIX taxon: <http://purl.uniprot.org/taxonomy/>
PREFIX up: <http://purl.uniprot.org/core/>
PREFIX void: <http://rdfs.org/ns/void#>
PREFIX lscr: <http://purl.org/lscr#>
PREFIX genex: <http://purl.org/genex#>

Select ?p ?species ?orthologGeneEns ?expr_breadth where {
{
  SELECT DISTINCT ?p ?species ?orthologGeneEns where { 
?p a orth:Protein.
?p orth:organism/obo:RO_0002162 taxon:559292.
?cluster a orth:OrthologsCluster.
?cluster orth:hasHomologousMember ?node1.
?cluster orth:hasHomologousMember ?node2. 
?node2 orth:hasHomologousMember* ?ortholog. 
?node1 orth:hasHomologousMember* ?p.
?ortholog sio:SIO_010079 ?gene . #is encoded by
?gene lscr:xrefEnsemblGene ?orthologGeneEns .
?ortholog orth:organism/obo:RO_0002162/up:scientificName ?species.
  
filter(?node1 != ?node2) 

} limit 100 }

{
  SERVICE <https://bgee.org/sparql/> {
			SELECT ?orthologGeneEns (count (distinct ?anat) as ?expr_breadth) {
				?geneB a orth:Gene .
				?geneB lscr:xrefEnsemblGene ?orthologGeneEns .
                ?expr <http://purl.org/genex#hasSequenceUnit> ?geneB.
                ?expr a <http://purl.org/genex#Expression> .
                ?expr genex:hasConfidenceLevel obo:CIO_0000029 . # high confidence level
                ?expr genex:hasExpressionLevel ?exprLevel .
                FILTER (?exprLevel > 99) # highly expressed      
                ?expr genex:hasExpressionCondition ?cond .
                ?cond genex:hasAnatomicalEntity ?anat .
			}  group by ?orthologGeneEns
            }
            } 
}
"""
sparql_endpoint_OMA.setQuery(query_limits)
sparql_endpoint_OMA.setReturnFormat(JSON)


results_groupby_Bgee = sparql_endpoint_OMA.query().convert()
#print(results)
#pretty_print(results)

EndPointInternalError: EndPointInternalError: endpoint returned code 500 and response. 

Response:
b"Virtuoso RDFZZ Error DB.DBA.SPARQL_REXEC('https://bgee.org/sparql/', ...) returned Content-Type 'text/plain' status 'HTTP/1.1 400 Bad Request\r\n'\nVirtuoso 37000 Error SP030: SPARQL compiler, line 12: syntax error at '}'\n\nSPARQL query:\ndefine sql:big-data-const 0  SELECT ?expr_breadth\n WHERE { \n      { SELECT  <http://rdf.ebi.ac.uk/resource/ensembl/ENSGACG00000014827>\n          ( COUNT( DISTINCT ?anat) AS ?expr_breadth)\n         WHERE {  ?geneB <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://purl.org/net/orth#Gene> ;\n                 <http://purl.org/lscr#xrefEnsemblGene> <http://rdf.ebi.ac.uk/resource/ensembl/ENSGACG00000014827> . ?expr <http://purl.org/genex#hasSequenceUnit> ?geneB ;\n                 <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://purl.org/genex#Expression> ;\n                 <http://purl.org/genex#hasConfidenceLevel> <http://purl.obolib

This initial attempt fails, because at query time, the values of *?orthologousGeneEns* are sent 1 by 1 from OMA to Bgee, hence there is no option to group by this anymore. In the error message above, you can see that in the Bgee service invocation the variable is already replaced by the URI <http://rdf.ebi.ac.uk/resource/ensembl/ENSGACG00000014827>.

### Second attempt: removing the groupby and simply counting elements in the remote query invocation (since the variable is replaced at query time already). 

This also fails, this time with a more cryptic Virtuoso error (Virtuoso HTCLI Error HC001: Read Error in HTTP Client):

In [206]:
%%time
query_limits = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX dc: <http://purl.org/dc/elements/1.1/>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX obo: <http://purl.obolibrary.org/obo/>
PREFIX ensembl: <http://rdf.ebi.ac.uk/resource/ensembl/>
PREFIX oma: <http://omabrowser.org/ontology/oma#>
PREFIX orth: <http://purl.org/net/orth#>
PREFIX sio: <http://semanticscience.org/resource/>
PREFIX taxon: <http://purl.uniprot.org/taxonomy/>
PREFIX up: <http://purl.uniprot.org/core/>
PREFIX void: <http://rdfs.org/ns/void#>
PREFIX lscr: <http://purl.org/lscr#>
PREFIX genex: <http://purl.org/genex#>

Select ?p ?species ?orthologGeneEns ?expr_breadth where {
{
  SELECT DISTINCT ?p ?species ?orthologGeneEns where { 
?p a orth:Protein.
?p orth:organism/obo:RO_0002162 taxon:559292.
?cluster a orth:OrthologsCluster.
?cluster orth:hasHomologousMember ?node1.
?cluster orth:hasHomologousMember ?node2. 
?node2 orth:hasHomologousMember* ?ortholog. 
?node1 orth:hasHomologousMember* ?p.
?ortholog sio:SIO_010079 ?gene . #is encoded by
?gene lscr:xrefEnsemblGene ?orthologGeneEns .
?ortholog orth:organism/obo:RO_0002162/up:scientificName ?species.
  
filter(?node1 != ?node2) 

} limit 100 }

{
  SERVICE <https://bgee.org/sparql/> {
			SELECT (count (distinct ?anat) as ?expr_breadth) {
				?geneB a orth:Gene .
				?geneB lscr:xrefEnsemblGene ?orthologGeneEns .
                ?expr <http://purl.org/genex#hasSequenceUnit> ?geneB.
                ?expr a <http://purl.org/genex#Expression> .
                ?expr genex:hasConfidenceLevel obo:CIO_0000029 . # high confidence level
                ?expr genex:hasExpressionLevel ?exprLevel .
                FILTER (?exprLevel > 99) # highly expressed      
                ?expr genex:hasExpressionCondition ?cond .
                ?cond genex:hasAnatomicalEntity ?anat .
			} 
            }
            } 
}
"""
sparql_endpoint_OMA.setQuery(query_limits)
sparql_endpoint_OMA.setReturnFormat(JSON)


results_no_groupby_Bgee = sparql_endpoint_OMA.query().convert()
#print(results)
#pretty_print(results)

EndPointInternalError: EndPointInternalError: endpoint returned code 500 and response. 

Response:
b'Virtuoso HTCLI Error HC001: Read Error in HTTP Client\n\nSPARQL query:\ndefine sql:big-data-const 0 \n#output-format:application/sparql-results+json\n\nPREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>\nPREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\nPREFIX owl: <http://www.w3.org/2002/07/owl#>\nPREFIX xsd: <http://www.w3.org/2001/XMLSchema#>\nPREFIX dc: <http://purl.org/dc/elements/1.1/>\nPREFIX dct: <http://purl.org/dc/terms/>\nPREFIX skos: <http://www.w3.org/2004/02/skos/core#>\nPREFIX obo: <http://purl.obolibrary.org/obo/>\nPREFIX ensembl: <http://rdf.ebi.ac.uk/resource/ensembl/>\nPREFIX oma: <http://omabrowser.org/ontology/oma#>\nPREFIX orth: <http://purl.org/net/orth#>\nPREFIX sio: <http://semanticscience.org/resource/>\nPREFIX taxon: <http://purl.uniprot.org/taxonomy/>\nPREFIX up: <http://purl.uniprot.org/core/>\nPREFIX void: <http://rdfs.org/ns/void#>\nPREFIX lscr:

### Attempted aggregation via SPARQL - failed....

###### **LESSON LEARNED no. 2: writing federated queries with aggregations is extremely challenging**

Given these errors, we abandon the obvious path of aggregating on the remote side and choose the inefficient (but feasible) path of retrieving data from the remote server and aggregating it (i.e. counting) locally. We cannot do this for the entirety of the orthologous genes as it would lead to a too large results set and likely timeouts, therefore we have to also limit the results in the first part of the query to have a chance of illustrating how this works:

We limit the results from OMA to the top 30K (protein, orthologous) pairs.

In [200]:
%%time
query_limits = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX dc: <http://purl.org/dc/elements/1.1/>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX obo: <http://purl.obolibrary.org/obo/>
PREFIX ensembl: <http://rdf.ebi.ac.uk/resource/ensembl/>
PREFIX oma: <http://omabrowser.org/ontology/oma#>
PREFIX orth: <http://purl.org/net/orth#>
PREFIX sio: <http://semanticscience.org/resource/>
PREFIX taxon: <http://purl.uniprot.org/taxonomy/>
PREFIX up: <http://purl.uniprot.org/core/>
PREFIX void: <http://rdfs.org/ns/void#>
PREFIX lscr: <http://purl.org/lscr#>
PREFIX genex: <http://purl.org/genex#>

Select ?p ?species ?orthologGeneEns ?anat where {
{
  SELECT DISTINCT ?p ?species ?orthologGeneEns where { 
?p a orth:Protein.
?p orth:organism/obo:RO_0002162 taxon:559292.
?cluster a orth:OrthologsCluster.
?cluster orth:hasHomologousMember ?node1.
?cluster orth:hasHomologousMember ?node2. 
?node2 orth:hasHomologousMember* ?ortholog. 
?node1 orth:hasHomologousMember* ?p.
?ortholog sio:SIO_010079 ?gene . #is encoded by
?gene lscr:xrefEnsemblGene ?orthologGeneEns .
?ortholog orth:organism/obo:RO_0002162/up:scientificName ?species.
  
filter(?node1 != ?node2) 

} limit 30000 }

{
  SERVICE <https://bgee.org/sparql/> {
			SELECT ?orthologGeneEns ?anat {
				?geneB a orth:Gene .
				?geneB lscr:xrefEnsemblGene ?orthologGeneEns .
                ?expr <http://purl.org/genex#hasSequenceUnit> ?geneB.
                ?expr a <http://purl.org/genex#Expression> .
                ?expr genex:hasConfidenceLevel obo:CIO_0000029 . # high confidence level
                ?expr genex:hasExpressionLevel ?exprLevel .
                FILTER (?exprLevel > 99) # highly expressed      
                ?expr genex:hasExpressionCondition ?cond .
                ?cond genex:hasAnatomicalEntity ?anat .
			}  
            }
            } 
}
"""
sparql_endpoint_OMA.setQuery(query_limits)
sparql_endpoint_OMA.setReturnFormat(JSON)


results = sparql_endpoint_OMA.query().convert()


CPU times: user 1.08 s, sys: 1.24 s, total: 2.32 s
Wall time: 51min 52s


In [209]:
pretty_print(results)

,p,species,orthologGeneEns,anat
0,https://omabrowser.org/oma/info/YEAST03651,Oryzias latipes,http://rdf.ebi.ac.uk/resource/ensembl/ENSORLG00000018367,http://purl.obolibrary.org/obo/UBERON_0000468
1,https://omabrowser.org/oma/info/YEAST03651,Oryzias latipes,http://rdf.ebi.ac.uk/resource/ensembl/ENSORLG00000018367,http://purl.obolibrary.org/obo/GO_0005575
2,https://omabrowser.org/oma/info/YEAST03651,Oryzias latipes,http://rdf.ebi.ac.uk/resource/ensembl/ENSORLG00000018367,http://purl.obolibrary.org/obo/UBERON_0000465
3,https://omabrowser.org/oma/info/YEAST03651,Oryzias latipes,http://rdf.ebi.ac.uk/resource/ensembl/ENSORLG00000018367,http://purl.obolibrary.org/obo/GO_0005575
4,https://omabrowser.org/oma/info/YEAST03651,Oryzias latipes,http://rdf.ebi.ac.uk/resource/ensembl/ENSORLG00000018367,http://purl.obolibrary.org/obo/UBERON_0000061
...,...,...,...,...
175511,https://omabrowser.org/oma/info/YEAST03807,Monodelphis domestica,http://rdf.ebi.ac.uk/resource/ensembl/ENSMODG00000017432,http://purl.obolibrary.org/obo/GO_0005575
175512,https://omabrowser.org/oma/info/YEAST03807,Monodelphis domestica,http://rdf.ebi.ac.uk/resource/ensembl/ENSMODG00000017432,http://purl.obolibrary.org/obo/UBERON_0000079
175513,https://omabrowser.org/oma/info/YEAST03807,Monodelphis domestica,http://rdf.ebi.ac.uk/resource/ensembl/ENSMODG00000017432,http://purl.obolibrary.org/obo/GO_0005575
175514,https://omabrowser.org/oma/info/YEAST03807,Monodelphis domestica,http://rdf.ebi.ac.uk/resource/ensembl/ENSMODG00000017432,http://purl.obolibrary.org/obo/UBERON_0003101


This query completed in 51m and produced 175K rows.

Now let us explore a bit the results. In particular, let's count for how many proteins we have data.

In [210]:
proteins = [entry["p"]["value"] for entry in results["results"]["bindings"]]
print(len(set(proteins)))

orthologs = [entry["orthologGeneEns"]["value"] for entry in results["results"]["bindings"]]
print(len(set(orthologs)))

35
507


### Compute aggregation directly in Python

Assuming we would have retrieved in the "results" variable all orthologs of S. cerevisiae in reference species contained in Bgee and the corresponding tissues where these orthologs are highly expressed, we could perform the subsequent aggregations offline (computing the number of orthologs per protein and the average expression breadth of these orthologs). Here we illustrate how this could be achieved, although results were truncated, therefore we will have only a partial view of the data.

In *results*, *p* is the protein in S. cerevisiae, *orthologGeneEns* is the orthologous protein (found in *species*) and *anat* is the tissue where the orthologous protein is highly expressed.

In [219]:
# create DataFrame with {Protein, Ortholog} pairs

# in the SPARQL results, data is retrieved in the "value" field of each entry in the "bindings" of the results
proteins_orthologs = [{'Protein_S_cerevisiae': entry["p"]["value"], 'Orthologous_Protein': entry["orthologGeneEns"]["value"]} for 
                      entry in results["results"]["bindings"]]

orthologs_df = pd.DataFrame.from_records(proteins_orthologs, columns=['Protein_S_cerevisiae', 'Orthologous_Protein'])
orthologs_df = orthologs_df.drop_duplicates()
orthologs_df["num_orthologs"] = orthologs_df.groupby("Protein_S_cerevisiae")["Orthologous_Protein"].transform("count")

In [220]:
num_orthologs_per_protein_sorted = orthologs_df[["Protein_S_cerevisiae","num_orthologs"]].drop_duplicates().sort_values("num_orthologs", ascending = False)

In [221]:
num_orthologs_per_protein_sorted

,Protein_S_cerevisiae,num_orthologs
23138,https://omabrowser.org/oma/info/YEAST03625,169
92428,https://omabrowser.org/oma/info/YEAST03653,45
396,https://omabrowser.org/oma/info/YEAST03641,40
61174,https://omabrowser.org/oma/info/YEAST03685,40
144892,https://omabrowser.org/oma/info/YEAST03626,34
70778,https://omabrowser.org/oma/info/YEAST03708,30
16866,https://omabrowser.org/oma/info/YEAST03687,29
47380,https://omabrowser.org/oma/info/YEAST03697,29
90810,https://omabrowser.org/oma/info/YEAST03706,15
167740,https://omabrowser.org/oma/info/YEAST03675,14


#### In how many species does each protein have orthologs?

In [242]:
# create DataFrame with {Protein, Species} pairs

# in the SPARQL results, data is retrieved in the "value" field of each entry in the "bindings" of the results
proteins_orthologs = [{'Protein_S_cerevisiae': entry["p"]["value"], 'Species_With_Orthologs': entry["species"]["value"]} for 
                      entry in results["results"]["bindings"]]

orthologs_species_df = pd.DataFrame.from_records(proteins_orthologs, columns=['Protein_S_cerevisiae', 'Species_With_Orthologs'])
orthologs_species_df = orthologs_species_df.drop_duplicates()

orthologs_species_df["num_species"] = orthologs_species_df.groupby("Protein_S_cerevisiae")["Species_With_Orthologs"].transform("count")

num_species_per_protein_sorted = orthologs_species_df[["Protein_S_cerevisiae","num_species"]].drop_duplicates().sort_values("num_species", ascending = False)

num_species_per_protein_sorted

,Protein_S_cerevisiae,num_species
92428,https://omabrowser.org/oma/info/YEAST03653,40
144892,https://omabrowser.org/oma/info/YEAST03626,33
396,https://omabrowser.org/oma/info/YEAST03641,29
70778,https://omabrowser.org/oma/info/YEAST03708,28
61174,https://omabrowser.org/oma/info/YEAST03685,28
23138,https://omabrowser.org/oma/info/YEAST03625,27
16866,https://omabrowser.org/oma/info/YEAST03687,26
47380,https://omabrowser.org/oma/info/YEAST03697,26
167740,https://omabrowser.org/oma/info/YEAST03675,14
90810,https://omabrowser.org/oma/info/YEAST03706,13


Similarly we want to compute the average expression breadth per protein, we can achieve this with the following:

In [223]:
# create DataFrame with {Protein, Ortholog, Tissue} pairs

# in the SPARQL results, data is retrieved in the "value" field of each entry in the "bindings" of the results
proteins_orthologs = [{'Protein_S_cerevisiae': entry["p"]["value"], 'Orthologous_Protein': entry["orthologGeneEns"]["value"],
                       'Tissue_where_Ortholog_expressed': entry["anat"]["value"]} for 
                      entry in results["results"]["bindings"]]

expr_breadth_df = pd.DataFrame.from_records(proteins_orthologs, columns=['Protein_S_cerevisiae', 'Orthologous_Protein', 'Tissue_where_Ortholog_expressed'])
expr_breadth_df = expr_breadth_df.drop_duplicates()
# now we want to count number of tissues per ortholog first
expr_breadth_df["number_of_tissues_per_ortholog"] = expr_breadth_df.groupby(["Protein_S_cerevisiae",'Orthologous_Protein'])["Tissue_where_Ortholog_expressed"].transform("count")

# after counting per ortholog, we want to average the expression breadth per protein

expr_breadth_df["average_expression_breadth"] = expr_breadth_df.groupby("Protein_S_cerevisiae")["number_of_tissues_per_ortholog"].transform("mean")

expr_breadth_df_sorted = expr_breadth_df[["Protein_S_cerevisiae","average_expression_breadth"]].drop_duplicates().sort_values("average_expression_breadth", ascending = False)

In [224]:
expr_breadth_df_sorted

,Protein_S_cerevisiae,average_expression_breadth
92428,https://omabrowser.org/oma/info/YEAST03653,194.037491
47380,https://omabrowser.org/oma/info/YEAST03697,166.221399
70778,https://omabrowser.org/oma/info/YEAST03708,132.368049
170320,https://omabrowser.org/oma/info/YEAST03845,132.228986
144892,https://omabrowser.org/oma/info/YEAST03626,124.314199
15316,https://omabrowser.org/oma/info/YEAST03693,121.752577
61174,https://omabrowser.org/oma/info/YEAST03685,119.032086
396,https://omabrowser.org/oma/info/YEAST03641,111.726172
23138,https://omabrowser.org/oma/info/YEAST03625,102.393535
167740,https://omabrowser.org/oma/info/YEAST03675,71.591195


### 6. Intersecting Top N from the two results to predict most likely candidates for essential genes

These will be the ones highly conserved across species (orthologs in many species), as well as a high expression breadth of these orthologs.

In [244]:
# select top N from each of the two dataframes above and join them

N = 10

top_N_expr = expr_breadth_df_sorted.head(N)

#top_N_ortho = num_orthologs_per_protein_sorted.head(N)

top_N_ortho = num_species_per_protein_sorted.head(N)
result = top_N_expr.set_index('Protein_S_cerevisiae').join(top_N_ortho.set_index('Protein_S_cerevisiae'))

In [245]:
result

,average_expression_breadth,num_species
Protein_S_cerevisiae,,
https://omabrowser.org/oma/info/YEAST03653,194.037491,40.0
https://omabrowser.org/oma/info/YEAST03697,166.221399,26.0
https://omabrowser.org/oma/info/YEAST03708,132.368049,28.0
https://omabrowser.org/oma/info/YEAST03845,132.228986,NaN
https://omabrowser.org/oma/info/YEAST03626,124.314199,33.0
https://omabrowser.org/oma/info/YEAST03693,121.752577,NaN
https://omabrowser.org/oma/info/YEAST03685,119.032086,28.0
https://omabrowser.org/oma/info/YEAST03641,111.726172,29.0
https://omabrowser.org/oma/info/YEAST03625,102.393535,27.0


Finally, we drop NaN rows and get the final candidates:

In [246]:
result = result.dropna()

In [247]:
result

,average_expression_breadth,num_species
Protein_S_cerevisiae,,
https://omabrowser.org/oma/info/YEAST03653,194.037491,40.0
https://omabrowser.org/oma/info/YEAST03697,166.221399,26.0
https://omabrowser.org/oma/info/YEAST03708,132.368049,28.0
https://omabrowser.org/oma/info/YEAST03626,124.314199,33.0
https://omabrowser.org/oma/info/YEAST03685,119.032086,28.0
https://omabrowser.org/oma/info/YEAST03641,111.726172,29.0
https://omabrowser.org/oma/info/YEAST03625,102.393535,27.0
https://omabrowser.org/oma/info/YEAST03675,71.591195,14.0


**Caveat**: retrieving only partial results will naturally result in only a partial view of the datasets involved, therefore the results should not be taken at face value, but rather experiments should be repeated without the *limit*s on the queries. Currently however this is not possible due to performance issues.